In [ ]:
# |default_exp gamepad

In [ ]:
# |export
import logging
from itertools import repeat
from random import randint
from time import sleep

from evdev import AbsInfo, UInput
from evdev import ecodes as e
from fastcore.basics import store_attr
from scipy.interpolate import interp1d

In [ ]:
# |export
axis_abs_info = AbsInfo(value=0, min=-32768, max=32767, fuzz=16, flat=128, resolution=0)
trigger_abs_info = AbsInfo(value=0, min=0, max=1023, fuzz=0, flat=0, resolution=0)
dpad_abs_info = AbsInfo(value=0, min=-1, max=1, fuzz=0, flat=0, resolution=0)
xbox_elite_uinput_args = dict(
    bustype=3,
    vendor=1118,
    product=2816,
    version=1032,
    events={
        e.EV_KEY: [
            e.BTN_A,
            e.BTN_B,
            e.BTN_X,
            e.BTN_Y,
            e.BTN_TL,
            e.BTN_TR,
            e.BTN_SELECT,
            e.BTN_START,
            e.BTN_MODE,
            e.BTN_THUMBL,
            e.BTN_THUMBR,
        ],
        e.EV_ABS: [
            (e.ABS_X, axis_abs_info),
            (e.ABS_Y, axis_abs_info),
            (e.ABS_RX, axis_abs_info),
            (e.ABS_RY, axis_abs_info),
            (e.ABS_HAT2Y, trigger_abs_info),
            (e.ABS_HAT2X, trigger_abs_info),
            (e.ABS_HAT0X, dpad_abs_info),
            (e.ABS_HAT0Y, dpad_abs_info),
        ],
    },
)

In [ ]:
# |export
class TmGamepad:
    axis_interp = interp1d([-1, 1], [axis_abs_info.min, axis_abs_info.max])
    trigger_interp = interp1d([0, 1], [trigger_abs_info.min, trigger_abs_info.max])
    binds = {
        "Accelerate": (e.EV_KEY, e.BTN_TR),
        "Brake": (e.EV_KEY, e.BTN_TL),
        "Steer": (e.EV_ABS, e.ABS_X),
        "Give up": (e.EV_KEY, e.BTN_B),
        "Respawn": (e.EV_KEY, e.BTN_Y),
        "Secondary respawn": (e.EV_KEY, e.BTN_X),
        "Show/Hide Interface": (e.EV_KEY, e.BTN_THUMBL),
        "Camera 3": (e.EV_KEY, e.BTN_THUMBR),
    }

    def __init__(self, name, tap_duration=0.1):
        store_attr()
        self.ui = UInput(name=name, phys=name, **xbox_elite_uinput_args)
        while self.ui.device is None:
            logging.debug(f"Gamepad {name} device is None, searching device again ...")
            self.ui.device = self.ui._find_device()
        logging.debug(f"Gamepad {name} path is {self.path}")

    @property
    def path(self):
        return self.ui.device.path

    def act(self, accelerate, brake, steer):
        self.ui.write(*self.binds["Accelerate"], int(accelerate))
        self.ui.write(*self.binds["Brake"], int(brake))
        self.ui.write(*self.binds["Steer"], int(self.axis_interp(steer)))
        self.ui.syn()

    def give_up(self):
        self.act(False, False, 0)
        self.tap_btn(self.binds["Give up"])

    def respawn(self, launched=True):
        self.act(False, False, 0)
        self.tap_btn(self.binds["Respawn" if launched else "Secondary respawn"])

    def toggle_interface(self):
        self.tap_btn(self.binds["Show/Hide Interface"])

    def tap_btn(self, btn):
        self.ui.write(e.EV_KEY, btn, 1)
        self.ui.syn()
        sleep(self.tap_duration)
        self.ui.write(e.EV_KEY, btn, 0)
        self.ui.syn()

    def tap_abs(self, abs):
        self.ui.write(e.EV_ABS, abs, axis_abs_info.max)
        self.ui.syn()
        sleep(self.tap_duration)
        self.ui.write(e.EV_ABS, abs, 0)
        self.ui.syn()

    def tap_a(self):
        self.tap_btn(e.BTN_A)

    def tap_b(self):
        self.tap_btn(e.BTN_B)

    def configure(self):
        for bind, (event_type, event) in self.binds.items():
            input(f'Open "{bind}" bind prompt ingame then press enter here ...')
            if event_type == e.EV_KEY:
                self.tap_btn(event)
            elif event_type == e.EV_ABS:
                self.tap_abs(event)
            else:
                raise TypeError(f"Unsupported event type {event_type}")

    def __del__(self):
        self.ui.close()

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()